In [1]:
import csv
import numpy as np
import decord
import torch
import os, time
import pandas as pd
import random

from gluoncv.torch.utils.model_utils import download
from gluoncv.torch.data.transforms.videotransforms import video_transforms, volume_transforms
from gluoncv.torch.engine.config import get_cfg_defaults
from gluoncv.torch.model_zoo import get_model
from tqdm.notebook import tqdm

map_path = 'action_map.csv'

action_dict = {}

with open(map_path, 'r', newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        id_value = int(row['id'])
        name_value = row['name']
        
        action_dict[id_value] = name_value

In [18]:
def process_frames(file_path, model, curr_class, N, strategy):
    vr = decord.VideoReader(file_path)
    new_rows = []
    INTERVAL = 60
    for time_int in range(0, len(vr) - INTERVAL + 1, INTERVAL):
        if strategy == "Random":
            l_bound, r_bound = time_int + N, (time_int + INTERVAL) - (N + 1)
            rand_start = random.randint(l_bound, r_bound)
            frame_id_list = range(rand_start-N, rand_start+N+1)
        elif strategy == "MostRecent":
            recent_start = (time_int + INTERVAL) - (N + 1)
            frame_id_list = range(recent_start-N, recent_start+N+1)
        elif strategy == "LeastRecent":
            lr_start = time_int + N
            frame_id_list = range(lr_start-N, lr_start+N+1)
        else:
            raise ValueError("Sampling strategy not yet supported")
        video_data = vr.get_batch(frame_id_list).asnumpy()
        crop_size = 224
        short_side_size = 256
        transform_fn = video_transforms.Compose([video_transforms.Resize(short_side_size, interpolation='bilinear'),
                                                video_transforms.CenterCrop(size=(crop_size, crop_size)),
                                                volume_transforms.ClipToTensor(),
                                                video_transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
        clip_input = transform_fn(video_data)
        with torch.no_grad():
            pred = model(torch.unsqueeze(clip_input, dim=0)).numpy()
        probs = torch.nn.functional.softmax(torch.tensor(pred), dim=1).numpy()
        top_class = np.argmax(probs)
        confidence_interval = np.max(probs) - np.min(probs)
        new_row = {"file_name": file_path.split("/")[-1], "class_id": curr_class, "class_name": action_dict[curr_class], "pred_class_id": top_class, "pred_class_name": action_dict[top_class], "window_size": N*2+1, "confidence": confidence_interval, "is_pred_correct": True if top_class == curr_class else False, "time_interval": time_int, "sampling_strategy": strategy}
        new_rows.append(new_row)
    return new_rows

In [19]:
# Get 5 Random videos from collection
dataset_path = 'datasets/kinetics400_5per/train'

subfolders = sorted([subfolder for subfolder in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, subfolder))])
random.seed(2)
random_subfolders = random.sample(subfolders, 5)
subfolder = random_subfolders[0]

config_file = './i3d_resnet50_v1_kinetics400.yaml'
cfg = get_cfg_defaults()
cfg.merge_from_file(config_file)
model = get_model(cfg)
model.eval()

columns = ["file_name", "class_id", "class_name", "pred_class_id", "pred_class_name", "window_size", "confidence", "is_pred_correct", "time_interval", "sampling_strategy"]

my_data = []

sampling_strategies = ['Random', 'MostRecent', 'LeastRecent']

try:
    # Iterate over videos
    for index, subfolder in tqdm(enumerate(random_subfolders), desc=f"Folder progress", position=0, total=len(random_subfolders)):
        subfolder_path = os.path.join(dataset_path, subfolder)
        files = os.listdir(subfolder_path)
        clean_files = []
        for filename in files:
            if filename.endswith(".mp4"):
                clean_files.append(filename)
        random_file = random.sample(clean_files, 1)[0]
        file_path = os.path.join(subfolder_path, random_file)
        class_id = -1
        for key, value in action_dict.items():
            if value == subfolder:
                class_id = key
                break
        N = 2
        # Do something with each file
        for N in tqdm(range(N, 10), desc=f"Context window progress for file: {filename}", position=2, leave=False):
            for strategy in sampling_strategies:
                new_rows = process_frames(file_path, model, class_id, N, strategy)
                for row in new_rows:
                    my_data.append(row)
                
except Exception as e:
    print(e)

finally:
    # Specify the file path
    out_path = 'sampling_strategies.csv'
    
    # Check if the file exists
    if os.path.exists(out_path):
        # Delete the file
        os.remove(out_path)
    df = pd.DataFrame(my_data, columns=columns)
    df.to_csv(out_path, mode='w', index=False)

Folder progress:   0%|          | 0/5 [00:00<?, ?it/s]

Context window progress for file: xX7KqiAim0k.mp4:   0%|          | 0/8 [00:00<?, ?it/s]

Context window progress for file: ITO1tntk1G8.mp4:   0%|          | 0/8 [00:00<?, ?it/s]

Context window progress for file: cHBkP4Gv8GM.mp4:   0%|          | 0/8 [00:00<?, ?it/s]

Context window progress for file: Kwh3zOUU-uY.mp4:   0%|          | 0/8 [00:00<?, ?it/s]

Context window progress for file: G8os8GeaX2k.mp4:   0%|          | 0/8 [00:00<?, ?it/s]